In [1]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import numpy as np

In [2]:
def coerce_position_vector(a):
    #if not isinstance(a, np.ndarray) or a.shape != (3, 1):
    if isinstance(a, (int, float)):
        a = np.array([[float(a), 0., 1.]]).T
    else:
        a = np.asarray(a).flatten()
        assert len(a) == 2 or len(a) == 3
        a = np.array([[a[0], a[1], 1.]]).T
    return a


def get_scale_matrix(x, y):
    return np.array([
        (x, 0., 0.),
        (0., y, 0.),
        (0., 0., 1.),
    ])


def get_rotation_matrix(angle):
    c = np.cos(angle)
    s = np.sin(angle)
    return np.array([
        (c, -s, 0.),
        (s, c, 0.),
        (0., 0., 1.),
    ])


def get_translation_matrix(offset):
    offset = coerce_position_vector(offset)
    return np.array([
        (1., 0., offset[0, 0]),
        (0., 1., offset[1, 0]),
        (0., 0., 1.),
    ])


def get_rotation_translation_matrix(angle, offset):
    offset = coerce_position_vector(offset)
    c = np.cos(angle)
    s = np.sin(angle)
    return np.array([
        (c, -s, offset[0, 0]),
        (s, c, offset[1, 0]),
        (0., 0., 1.),
    ])


def normalize_vector(vec):
    size = np.linalg.norm(vec)
    if np.isclose(size, 0.):
        vec = np.zeros(vec.shape)
    else:
        vec = vec / size
    return vec


def init_ax(ax, lim=10, hide_axes=True):
    ax.set_aspect('equal')
    ax.set_xlim(-lim * 16/9, lim * 16/9)
    ax.set_ylim(-lim, lim)

    if hide_axes:
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        #fig.patch.set_visible(False)
        ax.axis('off')
    else:
        ax.spines['bottom'].set_position('zero')
        ax.spines['left'].set_position('zero')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.xaxis.set_ticklabels([])
        ax.yaxis.set_ticklabels([])

In [3]:
scale = 1.

@interact(
    scale=(0.1, 5.),
    view_x=(-30, 30),
    view_y=(-30, 30),
    seed=(0, 100, 1),
    length=(0.1, 5.,0.1),
    q1=(-180, 180, 5),
    q2=(-180, 180, 5),
    q1d=(-3., 3., 0.1),
    q2d=(-3., 3., 0.1),
    deriv_frame1=[None, 0, 1, 2],
    deriv_frame2=[None, 0, 1, 2],
)
def f(
    scale=scale,
    view_x=0.,
    view_y=0.,
    q1=0.,
    q2=55.,
    mat=['mat1', 'mat2', 'mat3'],
    show_vectors=True,
    show_contour=True,
):
    q1 = q1 / 180 * np.pi
    q2 = q2 / 180 * np.pi    

    fig, ax0 = plt.subplots(1, 1, figsize=(16, 9))
    init_ax(ax0)
    xform_matrix = get_translation_matrix((-view_x, -view_y)).dot(get_scale_matrix(scale, scale))

    mat1 = get_rotation_matrix(q1)
    mat2 = get_rotation_matrix(q2)
    mat3 = mat1 + mat2
    if mat == 'mat1':
        mat = mat1
    elif mat == 'mat2':
        mat = mat2
    else:
        mat = mat3

    xs = np.linspace(-10., 10., 15)
    ys = np.linspace(-10., 10., 15)
    xg, yg = np.meshgrid(xs, ys)
    zg = np.ones(xg.shape)
    local_pos = np.array(np.row_stack((xg.ravel(), yg.ravel(), zg.ravel())))
    #global_pos = mat.dot(local_pos)
    global_pos = local_pos
    view_pos = xform_matrix.dot(global_pos)
    
    def do_contour(mat, color):
        if show_contour:
            ps = []
            for p in local_pos.T:
                ps.append(p.T.dot(mat).dot(p))                
            ax0.contour(view_pos[0].reshape(xg.shape), view_pos[1].reshape(yg.shape), np.array(ps).reshape(xg.shape), levels=6, colors=color)
    
    def do_quiver(mat, color):
        if show_vectors:
            global_vel = mat.dot(local_pos)
            view_vel = xform_matrix.dot(global_vel)
            ax0.quiver(view_pos[0], view_pos[1], view_vel[0], view_vel[1], width=0.002, color=color, alpha=0.3)

    do_quiver(mat, 'green')
    do_contour(mat.T.dot(mat), 'green')

interactive(children=(FloatSlider(value=1.0, description='scale', max=5.0, min=0.1), IntSlider(value=0, descri…